Check co-evolution of lambda, halo mass, stellar mass. 

In [1]:
import pickle
# Galaxies of interest
#mpgs = pickle.load(open("main_prgs_final_augmented_5_10_0.5_0.5_0.5_37_0.01_filtered_.pickle", "rb"))
g29176, Maj1 = pickle.load(open("g29176_Maj1.pickle", "rb"))

In [2]:
Majs = [mpgs[728], mpgs[226], mpgs[742]]

selected_cluster = 29176
selected_gals = [15, 17] # minor mergers
Mins = [gal for gal in mpgs if (gal.cluster == selected_cluster and gal.ids[0] in selected_gals)]

selected_gals = [45, 50, 98, 130] # no mergers
rests = [gal for gal in mpgs if (gal.cluster == selected_cluster and gal.ids[0] in selected_gals)]

In [6]:
g29176=[Majs[0], Majs[2], Mins[0], Mins[1], rests[0], rests[1], rests[2], rests[3]]
#pickle.dump([g29176, Majs[1]], open("g29176_Maj1.pickle", "wb"))

In [4]:
# Load halo and find matches


Hall=[]
for nout in range(37, 188)[:2]:
    h_now = []
    hh = hmo.Halo(base='./29176/', nout=nout, is_gal=False)
    
    h_now.append()

In [7]:
# find closest halos.
gal = g29176[0]

In [56]:
import numpy as np
from tree.ctutils import extract_main_tree, extract_a_tree
from tree import halomodule as hmo
import matplotlib.pyplot as plt

In [4]:
tt = pickle.load(open("./29176/halo/Trees/extended_tree.pickle", "rb"))    
ttf = tt.data[tt.data["nout"] == 187]

In [8]:
ttf = tt.data[tt.data["nout"] == 187]

In [63]:
def i_close_10(pos, hx,hy,hz):
    dist = np.sqrt(np.square(hx-pos[0]) + 
                   np.square(hy-pos[1]) + 
                   np.square(hz-pos[2]))
    i = np.argsort(dist)[:10]
    return dist[i], i

In [ ]:
tree_at_fi=[]
hh = hmo.Halo(base='./29176/', nout=187, is_gal=False)
hx = hh.data["x"] * hh.info.pboxsize
hy = hh.data["y"] * hh.info.pboxsize
hz = hh.data["z"] * hh.info.pboxsize

A main tree is not enough. 
The DM halo of a galaxy may have been desolved already. 
If the real halo is not detected, then there is no point selecting the closest halo to be the host. Instead, selected the locally dominant halo and a branch of the halo tree may be the real progenitor host halo of the galaxy. 

In [64]:
#ind_candidates=[]
idx_trees=[]
inow = 0 # 187
dist_trees=[]
for gal in g29176:
    pos_now = [gal.data["xc"][inow],
               gal.data["yc"][inow],
               gal.data["zc"][inow]] # in pboxsize
    dist_candidates, ind_candidates = i_close_10(pos_now, hx,hy,hz)
    
    inow = 0
    idx_tree_this=[]
    dist_this=[]
    for dd, ii in zip(dist_candidates, ind_candidates):
        i = np.where(ttf["Orig_halo_id"] == hh.data["id"][ii])[0]
        if len(i) > 0:
            # More massive than the galaxy stellar mass
            if ttf["m"][i] > gal.data["mstar"][inow]:
                idx_tree_this.append(ttf['id'][i][0].astype(int))
                dist_this.append(dd)
    idx_trees.append(idx_tree_this)
    dist_trees.append(dist_this)

In [65]:
for idx, dd in zip(idx_trees, dist_trees):
    print(idx, dd)

[942065, 942082, 942635, 941854] [0.0012646398, 0.45553318, 0.50928724, 0.52071685]
[942591, 942590] [0.00065064913, 0.18535948]
[942372] [0.00091403205]
[942392] [0.0015397207]
[942843, 942847, 941686, 942845, 942816] [0.0002863819, 0.20636073, 0.21796507, 0.26052681, 0.39437816]
[942337, 942326, 942403] [0.00276988, 0.14250007, 0.24111241]
[942715, 941860, 941814, 942697, 941726, 941815, 942700] [0.00041262261, 0.32559675, 0.43446591, 0.47934696, 0.51713967, 0.57483423, 0.57964987]
[941749, 941877, 941742, 940802] [0.0016048047, 0.4683924, 0.49877271, 0.62962329]


In [67]:
idx_good=[942065, 942591, 942372,
          942392, 942843, 942337,
          942715, 941749]

In [58]:
atree = extract_a_tree(tt.data, 942392)

In [84]:
from analysis.all_plot_modules import *

In [68]:
tts = []
for ii in idx_good:
    tts.append(extract_a_tree(tt.data, ii))

In [90]:
from matplotlib.backends.backend_pdf import PdfPages
fig, ax = plt.subplots(4, sharex=True)
plt.subplots_adjust(hspace=0.001)
with PdfPages('lambda_mhalo_mstar4.pdf') as pdf:   
    for igal in range(len(g29176)):
        i_nout_ok = np.where(tts[igal]["nout"] > min(g29176[igal].nouts))[0]
        gals = g29176[igal:igal+1]
        if igal < 2:            
            plot_major(gals, ax[0],
                   suptitle="Major Mergers",
                   img_scale=1.5,
                   arrow_scale=20) # arrow_scale = 50 for png, 20 for vectors.
        elif igal < 5:
            plot_minor(gals, ax[0],
                          suptitle="Minor Mergers",
                          style="stream",
                          img_scale=1.5,
                          annotate="(B) ",
                          arrow_scale=20)
        else:    
            plot_rest(gals, ax[0],
                  suptitle="No Mergers",
                  style="stream",
                  img_scale=1.5,
                  annotate="(C) ",
                  arrow_scale=20)
        
        ax[0].set_ylim([0,0.8])
        
        if 1 == 2:
            ax[0].scatter(tts[igal]["nout"][i_nout_ok],
                          tts[igal]["x"][i_nout_ok] * tts[igal]["aexp"][i_nout_ok] / 0.704,
                          label="tree")
            ax[0].plot(g29176[igal].nouts, g29176[igal].data["xc"], label="gal")
            ax[0].scatter(tts[igal]["nout"][i_nout_ok],
                          tts[igal]["y"][i_nout_ok]*tts[igal]["aexp"][i_nout_ok] / 0.704,
                          label="tree")
            ax[0].plot(g29176[igal].nouts, g29176[igal].data["yc"], label="gal")
            ax[0].scatter(tts[igal]["nout"][i_nout_ok],
                          tts[igal]["z"][i_nout_ok]*tts[igal]["aexp"][i_nout_ok] / 0.704,
                          label="tree")
            ax[0].plot(g29176[igal].nouts, g29176[igal].data["zc"], label="gal")
            #ax[0].legend()
        #else:
        #    ax[0].plot(g29176[igal].nouts, g29176[igal].smoothed_lambda)
        #    ax[0].set_ylim([0., 0.7])
        
        ax[1].scatter(tts[igal]["nout"][i_nout_ok],
                      np.log10(tts[igal]["m"][i_nout_ok]),
                      edgecolor="none", s=6, alpha=0.5,
                      label="tree")
        ax[1].plot(g29176[igal].nouts, np.log10(g29176[igal].data["mstar"]), label="gal")
        #ax[1].legend()
        
        ax[2].scatter(tts[igal]["nout"][i_nout_ok],
                      tts[igal]["vx"][i_nout_ok],
                      edgecolor="none", s=6, alpha=0.5,
                      label="tree")
        ax[2].plot(g29176[igal].nouts, g29176[igal].data["vxc"], label="gal")
        #ax[2].legend()
        ax[3].scatter(tts[igal]["nout"][i_nout_ok],
                      tts[igal]["vy"][i_nout_ok],
                      edgecolor="none", s=6, alpha=0.5,
                      label="tree")
        ax[3].plot(g29176[igal].nouts, g29176[igal].data["vyc"], label="gal")
        
        pdf.savefig()
        ax[0].clear()
        ax[1].clear()
        ax[2].clear()
        ax[3].clear()
    plt.close()

In [70]:
len(tts[0])

21

In [20]:

plt.hist(dist)
plt.show()

In [24]:
plt.scatter(hx[i_close5],hy[i_close5])
plt.scatter(pos_now[0], pos_now[1], c="r")
plt.show()

In [15]:
pos_now

[38.819926721406908, 57.281819347391654, 28.697416528931843]